In [12]:
import os
import pandas as pd
import cv2

In [13]:
ROOT_DIR = os.path.join(os.getcwd(), os.pardir)
DATA_DIR = os.path.join(ROOT_DIR, 'data')
TRAIN_VIDEOS_DIR = os.path.join(DATA_DIR, 'train_videos')
MARKUP_DIR = os.path.join(DATA_DIR, 'markup_df_train.xlsx')
VIDEOS_DIR = os.path.join(DATA_DIR, 'train_videos')

print(MARKUP_DIR)
print(VIDEOS_DIR)

/home/dead4an/traffic-violations-detection/notebooks/../data/markup_df_train.xlsx
/home/dead4an/traffic-violations-detection/notebooks/../data/train_videos


In [14]:
video_paths = []

# get video paths
for filename in os.listdir(TRAIN_VIDEOS_DIR):
    # ignore txt
    if filename.endswith('.txt'):
        continue

    video_path = os.path.join(TRAIN_VIDEOS_DIR, filename)
    video_paths.append(video_path)

In [15]:
violation_df = pd.DataFrame(columns=['id', 'time', 'violation'])
for filename in video_paths:
    # захватываем видео
    video = cv2.VideoCapture(filename)

    # считаем продолжительность в секундах
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = round(video.get(cv2.CAP_PROP_FPS))
    duration = round(total_frames / fps)

    # получаем имя файла и обрезаем расширение
    filename = filename.split(os.path.sep)[-1]
    filename = filename[:filename.find('.')]

    # создаем df для этого видео
    video_df = pd.DataFrame({'id': filename.split(os.path.sep)[-1].lower(), 
                             'time': range(1, duration + 1),
                             'violation': 'nothing'})
    
    violation_df = pd.concat((violation_df, video_df), axis=0)

In [16]:
violation_df

,id,time,violation
0,akn00036,1,nothing
1,akn00036,2,nothing
2,akn00036,3,nothing
3,akn00036,4,nothing
4,akn00036,5,nothing
...,...,...,...
295,akn00080,296,nothing
296,akn00080,297,nothing
297,akn00080,298,nothing
298,akn00080,299,nothing


In [17]:
# Чтение файла Excel в DataFrame
markup_df = pd.read_excel(MARKUP_DIR)
markup_df.head()

,номер видео,наименование нарушения,"сумма штрафа, руб.",время нарушения (в секундах)
0,akn00006_fqGg6dtL,"Статья 12.16. часть 1 Несоблюдение требований,...",500,18
1,akn00006_fqGg6dtL,Статья 12.16 часть 2 Поворот налево или развор...,1000,75
2,akn00006_fqGg6dtL,"Статья 12.16. часть 1 Несоблюдение требований,...",500,249
3,akn00006_fqGg6dtL,Статья 12.17 часть 1.1 и 1.2. движение трансп...,1500,250
4,akn00006_fqGg6dtL,"Статья 12.16. часть 1 Несоблюдение требований,...",500,263


In [18]:
for idx, row in markup_df.iterrows():
    video_id = row.iloc[0].lower() # номер видео
    violation = row.iloc[1] # наименование нарушения
    violation_time = row.iloc[3] # время нарушения

    # получаем продолжительность видео
    video_duration = violation_df[violation_df['id'] == video_id]['time'].max()
    window_start = max(1, violation_time - 10) # начало временного окна в котором ищем нарушение
    window_end = min(video_duration, violation_time + 10) # конец временного окна в котором ищем нарушение

    # выбираем записи с данным video_id, временное окно и устанавливаем наименование нарушения
    mask = violation_df['id'] == video_id
    violation_df.loc[mask & (violation_df.index >= window_start) & (violation_df.index < window_end), 'violation'] = violation

In [19]:
violation_df

,id,time,violation
0,akn00036,1,nothing
1,akn00036,2,Статья 12.12 часть 2 1. невыполнение требовани...
2,akn00036,3,Статья 12.12 часть 2 1. невыполнение требовани...
3,akn00036,4,Статья 12.12 часть 2 1. невыполнение требовани...
4,akn00036,5,Статья 12.12 часть 2 1. невыполнение требовани...
...,...,...,...
295,akn00080,296,nothing
296,akn00080,297,nothing
297,akn00080,298,nothing
298,akn00080,299,nothing


In [20]:
violation_df[violation_df['violation'] != 'nothing']

,id,time,violation
1,akn00036,2,Статья 12.12 часть 2 1. невыполнение требовани...
2,akn00036,3,Статья 12.12 часть 2 1. невыполнение требовани...
3,akn00036,4,Статья 12.12 часть 2 1. невыполнение требовани...
4,akn00036,5,Статья 12.12 часть 2 1. невыполнение требовани...
5,akn00036,6,Статья 12.12 часть 2 1. невыполнение требовани...
...,...,...,...
61,akn00080,62,Статья 12.16 часть 2 Поворот налево или развор...
62,akn00080,63,Статья 12.16 часть 2 Поворот налево или развор...
63,akn00080,64,Статья 12.16 часть 2 Поворот налево или развор...
64,akn00080,65,Статья 12.16 часть 2 Поворот налево или развор...
